## List the rasters

In [5]:
# listrasters.py
import os
import arcpy
from arcpy import env
env.workspace = os.getcwd() + "\\DATA"

In [6]:
rasterlist = arcpy.ListRasters()
for raster in rasterlist:
    print raster

elevation
landcover.tif
tm.img


## Describe the rasters

In [8]:
# describerasters.py
import os
import arcpy
from arcpy import env
env.workspace = os.getcwd() + "\\DATA"

In [11]:
# raster = "tm.img"
raster = "landcover.tif"
desc = arcpy.Describe(raster)
print "Raster base name is: " + desc.basename
print "Raster data type is: " + desc.dataType
print "Raster file extension is: " + desc.extension

Raster base name is: landcover
Raster data type is: RasterDataset
Raster file extension is: tif


In [12]:
print "Raster spatial reference is: " + desc.spatialReference.name
print "Raster format is: " + desc.format
print "Raster compression type is: " + desc.compressionType
print "Raster number of bands is: " + str(desc.bandCount)

Raster spatial reference is: Projection_Name_Albers_Conical_Equal_Area_Units_meters_GeoTIFF
Raster format is: TIFF
Raster compression type is: None
Raster number of bands is: 1


---

In [13]:
# describerasters2.py
import os
import arcpy
from arcpy import env
env.workspace = os.getcwd() + "\\DATA"

In [14]:
raster = "landcover.tif"
desc = arcpy.Describe(raster)
x = desc.meanCellHeight
y = desc.meanCellWidth
spatialref = desc.spatialReference
units = spatialref.linearUnitName
print "The raster resolution is " + str(x) + " by " + str(y) + " " + units + "."

The raster resolution is 30.0 by 30.0 Meter.


---

In [15]:
# describerasters2.py
import os
import arcpy
from arcpy import env
env.workspace = os.getcwd() + "\\DATA"

In [16]:
rasterband = "tm.img/Layer_1"
desc = arcpy.Describe(rasterband)
x = desc.meanCellHeight
y = desc.meanCellWidth
spatialref = desc.spatialReference
units = spatialref.angularUnitName
print "The raster resolution of Band 1 is " + str(x) + " by " + str(y) + " " + units + "."

The raster resolution of Band 1 is 0.000277778 by 0.000277778 Degree.


## Use raster objects in geoprocessing

In [19]:
# for python window
import os
import arcpy
from arcpy import env
env.workspace = os.getcwd() + "\\DATA"

In [ ]:
outraster = arcpy.sa.Slope("elevation")
desc = arcpy.Describe(outraster)
print desc.permanent

In [ ]:
outraster.save("Results/slope")

In [ ]:
from arcpy.sa import *
outraster2 = Aspect("elevation")

In [ ]:
outraster2.save("Results/aspect")

## Use map algebra operators

In [10]:
import os
import arcpy
from arcpy import env
env.workspace = os.getcwd() + "\\DATA"

In [ ]:
elevraster = arcpy.Raster("elevation")
outraster3 = elevraster * 3.281
outraster3.save("Results/elev_ft")

In [ ]:
slope = arcpy.sa.Slope(elevraster)
goodslope = slope < 20
goodelev = elevraster < 2000
goodfinal = goodslope & goodelev
goodfinal.save("Results/final")

## Work with classes to defie raster tool parameters

In [14]:
# reclass.py
import os
import arcpy
from arcpy import env
from arcpy.sa import *
env.workspace = os.getcwd()

In [15]:
if arcpy.CheckExtension("spatial") == "Available":
    arcpy.CheckOutExtension("spatial")
    #--
    outraster = arcpy.sa.Slope("DATA/elevation", "PERCENT_RISE")
    outraster.save("Results/slope_per")
    #--
    arcpy.CheckInExtension("spatial")
else:
    print "Spatial Analyst license is not available."

---

In [16]:
if arcpy.CheckExtension("spatial") == "Available":
    arcpy.CheckOutExtension("spatial")
    #--
    myremap = RemapRange([[1000,2000,1], [2000,3000,2], [3000,4000,3]])
    outreclass = Reclassify("DATA/elevation", "VALUE", myremap)
    outreclass.save("Results/elev_recl")
    #--
    arcpy.CheckInExtension("spatial")
else:
    print "Spatial Analyst license is not available."

---

In [17]:
if arcpy.CheckExtension("spatial") == "Available":
    arcpy.CheckOutExtension("spatial")
    myremap = RemapValue([[41,1], [42,2], [43,3]])
    outreclass = Reclassify("DATA/landcover.tif", "VALUE", myremap, "NODATA")
    outreclass.save("Results/lc_recl")
    arcpy.CheckInExtension("spatial")
else:
    print "Spatial Analyst license is not available."

---

In [18]:
# neighborhood.py
import os
import arcpy
from arcpy import env
from arcpy.sa import *
env.workspace = os.getcwd()

In [20]:
if arcpy.CheckExtension("spatial") == "Available":
    arcpy.CheckOutExtension("spatial")
    mynbr = NbrCircle(3, "CELL")
    outraster = FocalStatistics("DATA/landcover.tif", mynbr, "MAJORITY")
    outraster.save("Results/lc_nbr")
    arcpy.CheckInExtension("spatial")
else:
    print "Spatial Analyst license is not available."

## Challenge exercises

### Challenge 1

```
Create a script that determines what areas meet the following
conditions:
• Moderate slope—between 5 and 20 degrees
• Southerly aspect—between 150 and 270 degrees
• Forested—land cover types of 41, 42, or 43
Be sure to use the map algebra expressions of the arcpy.sa module.
```

In [22]:
import os
import arcpy
from arcpy import env
from arcpy.sa import *
env.workspace = os.getcwd()

In [24]:
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
elev = arcpy.Raster("DATA/elevation")
lc = arcpy.Raster("DATA/landcover.tif")
slope = Slope(elev)
aspect = Aspect(elev)
goodslope = ((slope > 5) & (slope < 20))
goodaspect = ((aspect > 150) & (aspect < 270))
goodland = ((lc == 41) | (lc == 42) | (lc ==43))
outraster = (goodslope & goodaspect & goodland)
outraster.save("Results/final_cha1")
arcpy.CheckInExtension("Spatial")

u'CheckedIn'

### Challenge 2

```
Write a script that copies all the rasters in a workspace to a new file geodatabase.
You can use the rasters in the Exercise09 folder as an example.
```

In [41]:
import os
import arcpy
from arcpy import *
out_path = os.getcwd()
env.workspace = out_path + "\\DATA"

In [46]:
rasterlist = arcpy.ListRasters()
arcpy.CreatePersonalGDB_management(out_path + "\\Results", "myrasters.mdb")
for raster in rasterlist:
    desc = arcpy.Describe(raster)
    rname = desc.baseName
    outraster = out_path + "\\Results\\myrasters.mdb\\" + rname
    arcpy.CopyRaster_management(raster, outraster)